# Libs

In [2]:
import warnings
warnings.filterwarnings('ignore', category=DeprecationWarning)
warnings.filterwarnings('ignore', category=UserWarning)

import psycopg2 
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from dotenv import load_dotenv
import os

load_dotenv()

True

# Connection

In [3]:
USER = os.environ.get('POSTGRES_USER')
PASSWORD = os.environ.get('POSTGRES_PASSWORD')
DB = os.environ.get('POSTGRES_DB_stepik')

conn_string = psycopg2.connect(f"dbname={DB} user='{USER}' host='localhost' password='{PASSWORD}'")
cursor = conn_string.cursor()
cursor.execute("SELECT version();")
record = cursor.fetchone()
print(f"Подключение к {record} выполнено")

Подключение к ('PostgreSQL 15.8 (Debian 15.8-1.pgdg120+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 12.2.0-14) 12.2.0, 64-bit',) выполнено


# 1. Data selection

In [64]:
# 1. Вывести информацию о всех книгах, хранящихся на складе.
q1 = ('  select * from book')
# 2. Выбрать названия книг и авторов из таблицы book, для поля title задать имя(псевдоним) Название, для поля author –  Автор.
q2 = ('  select title as Название, author as Автор from book')
# 3. Для упаковки каждой книги требуется один лист бумаги, цена которого 1 рубль 65 копеек. Посчитать стоимость упаковки для каждой книги 
# (сколько денег потребуется, чтобы упаковать все экземпляры книги). В запросе вывести название книги, ее количество и стоимость упаковки, 
# последний столбец назвать pack. 
q3 = (' select title, amount, amount * 1.65 as pack \
        from book')
# 4. В конце года цену каждой книги на складе пересчитывают – снижают ее на 30%. Написать SQL запрос, который из таблицы book выбирает названия, авторов, количества и 
# вычисляет новые цены книг. Столбец с новой ценой назвать new_price, цену округлить до 2-х знаков после запятой.
q4 = (' select title, author, amount, \
        round((price*0.7),2) as new_price \
        from book')
# 5. При анализе продаж книг выяснилось, что наибольшей популярностью пользуются книги Михаила Булгакова, на втором месте книги Сергея Есенина. 
# Исходя из этого решили поднять цену книг Булгакова на 10%, а цену книг Есенина - на 5%. Написать запрос, куда включить автора, название книги и новую цену, 
# последний столбец назвать new_price. Значение округлить до двух знаков после запятой.
q5 = (  'select author, title, '
        'round( '
        '       case '
        '           when author = \'Булгаков М.А.\' then price * 1.1 '
        '           when author = \'Есенин С.А.\' then price * 1.05 '
        '           else price '
        '       end, 2) as new_price '
        'from book')
# 6. Вывести автора, название  и цены тех книг, количество которых меньше 10.
q6 = (' select author, title, price \
        from book where amount < 10')
# 7. Вывести название, автора,  цену  и количество всех книг, цена которых меньше 500 или больше 600, 
# а стоимость всех экземпляров этих книг больше или равна 5000.
q7 = (' select title, author, price, amount \
        from book \
        where (price < 500 OR price > 600) and price*amount >= 5000')
# 8. Вывести название и авторов тех книг, цены которых принадлежат интервалу от 540.50 до 800 (включая границы),  
# а количество или 2, или 3, или 5, или 7.
q8 = (' select title, author \
        from book \
        where price between 540.5 and 800 and amount in (2, 3, 5, 7)')
# 9. Вывести  автора и название  книг, количество которых принадлежит интервалу от 2 до 14 (включая границы). 
# Информацию  отсортировать сначала по авторам (в обратном алфавитном порядке), а затем по названиям книг (по алфавиту).
q9 = (' select author, title \
        from book \
        where amount between 2 and 14 \
        order by author desc, title asc')
# 10. Вывести название и автора тех книг, название которых состоит из двух и более слов, а инициалы автора содержат букву «С». 
# Считать, что в названии слова отделяются друг от друга пробелами и не содержат знаков препинания, между фамилией автора 
# и инициалами обязателен пробел, инициалы записываются без пробела в формате: буква, точка, буква, точка. 
# Информацию отсортировать по названию книги в алфавитном порядке.
q10 = ('select title, author from book \
        where title like \'_% _%\' and author like \'%С.%\' \
        order by title asc')
# 11. Магазин счёл, что классика уже не пользуется популярностью, поэтому необходимо в выборке:
# Сменить всех авторов на "Донцова Дарья".
# К названию каждой книги в начале дописать "Евлампия Романова и".
# Цену поднять на 42%.
# Отсортировать по убыванию цены и убыванию названия.
q11 = ( 'select \'Донцова Дарья\' as author,' + 
        'concat(\'Евлампия Романова и \' , title) as title, ' +
        'round((price * 1.42),2) as price '+
        'from book order by price desc, title desc')

data = pd.read_sql(q11, con=conn_string)
data

,author,title,price
0,Донцова Дарья,Евлампия Романова и Братья Карамазовы,1134.59
1,Донцова Дарья,Евлампия Романова и Мастер и Маргарита,952.81
2,Донцова Дарья,Евлампия Романова и Стихотворения и поэмы,923.00
3,Донцова Дарья,Евлампия Романова и Белая гвардия,767.51
4,Донцова Дарья,Евлампия Романова и Игрок,682.31
5,Донцова Дарья,Евлампия Романова и Идиот,653.20
